# 📓 3_Explainability.ipynb

Apply SHAP to understand model predictions on Photosensitivity Risk.

In [ ]:
# Step 1: Import libraries
import torch
import shap
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Step 2: Load preprocessed data
# Make sure preprocessed_data.csv exists in your environment!
data = pd.read_csv('data/preprocessed/preprocessed_data.csv')

X = data[['brightness_level', 'eye_movement_variance']].values
y = data['photosensitivity_risk'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
# Step 3: Define and load the model
class DualBranchNet(torch.nn.Module):
    def __init__(self):
        super(DualBranchNet, self).__init__()
        self.fc1 = torch.nn.Linear(2, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.fc3 = torch.nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Initialize and load trained weights
model = DualBranchNet()
model.load_state_dict(torch.load('dual_branch_cnn.pth'))
model.eval()

In [ ]:
# Step 4: Apply SHAP Explainability (Corrected)
# Convert tensor to numpy
X_test_np = X_test_tensor.detach().numpy()

# Define predict function for SHAP
def predict_fn(x):
    x_tensor = torch.tensor(x, dtype=torch.float32)
    with torch.no_grad():
        return model(x_tensor).numpy()

# KernelExplainer for model
explainer = shap.KernelExplainer(predict_fn, X_test_np[:50])  # Small background set for speed
shap_values = explainer.shap_values(X_test_np[:100])          # Explain 100 samples

# Plot SHAP values
shap.summary_plot(shap_values, X_test_np[:100], feature_names=['brightness_level', 'eye_movement_variance'])

## 🎯 Congratulations!

You've now:
- Loaded your model
- Applied SHAP Explainability properly
- Visualized how brightness and eye movements contribute to risk prediction!

✅ Notebook is metadata-clean and ready to upload to GitHub.